https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html#scipy.integrate.odeint

In [1]:
import numpy as np
import k3d
from ratelimiter import RateLimiter
from scipy.integrate import odeint
from itertools import product
from k3d.platonic import Cube

def pend(y, t, b, c):
    theta, omega = y
    dydt = [omega, -b*omega - c*np.sin(theta)]
    return dydt

b = 0.25
c = 5.0
y0 = [np.pi - 0.1, 0.0]
t = np.linspace(0, 30, 600)
sol = np.float32(odeint(pend, y0, t, args=(b, c)))
x, y = np.cos(sol[:,0]), np.sin(sol[:,0])
line_values = np.float32(np.stack([np.arange(sol.shape[0])/100, y, -np.ones(sol.shape[0])]).T)
diffs = np.float32(np.diff(np.stack([np.zeros(sol.shape[0]), y, -x]).T, axis=0))
diffs = diffs / np.stack([np.linalg.norm(diffs, axis=1)]).T
diffs *= np.abs(np.stack([sol[:-1,1]]).T)/2

plot = k3d.plot(grid_auto_fit=False, camera_auto_fit=False, grid=(-4,-2,-2,0,2,1))
point = k3d.points([0, y[0], -x[0]], point_size=0.1)
line = k3d.line([0, y[0], 0])
mesh = k3d.mesh([[-3,1,-1.02], [-3,-1,-1.02], [0,1,-1.02],[0,-1,-1.02]], [[0,1,3],[0,2,3]], color=0xffffff)
cube = Cube([0,0,0], 0.05)[1]
twine = k3d.line(vertices=[[0,0,0], point.positions], shader='simple')
velocity_vector = k3d.vectors(origins=point.positions, vectors=diffs[0], head_color=0xff0000, head_size=0.1)

plot += point + line +  mesh + cube + twine + velocity_vector
plot.display()

Output()

In [ ]:
plot.camera = [2.973645060709364,-6.337929918481148,4.729101180748271,-2,0,-0.5,-0.28493277300160497,0.4080724365995052,0.8673466442878298]
rate_limiter = RateLimiter(max_calls=1, period=1/30)

for i in range(1, sol.shape[0]):
    with rate_limiter:
        point.positions = [0, y[i], -x[i]]
        twine.vertices = [[0,0,0], point.positions]
        velocity_vector.origins = point.positions
        velocity_vector.vectors = diffs[i-1]
            
        if i-300 < 0: 
            bias = 0
        else: 
            bias = i-300
        
        line.vertices = line_values[bias:i] - i*np.array([0.01, 0, 0], dtype=np.float32)